# CSCI 3220: Python for Data Science and AI

## Assignment 3

## Due Date: April 11(Monday), 11:59 PM

## Total Points: 35

## Bonus Points: 5

#### ***Write the code in Jupyter notebook (ipynb file) with proper comments. 
#### ***Add proper citation if you take help from a different source (not from the textbook).
#### ***Rename the file with your student ID and submit it in Moodle.

In [2]:
import pandas as pd
import numpy as np
import sklearn

## 1a.	Load any one of the toy datasets (for classification) from scikit-learn library.

In [12]:
from sklearn.datasets import load_iris

iris = load_iris()

## 1b. Display general information and statistics about your selected dataset.

In [10]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## 2.	Split the dataset into train and test sets.

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target)

## 3.	Implement a k-nearest neighbors classifier (k =3) for the training set. Show the result for the test set using the following metric: accuracy, precision, and recall.

## 4.	Split the training set into 10-fold for cross-validation. Calculate accuracy for each held-out set. Also, display the average accuracy for ten folds.

## 5.	Find the optimal value of k ranging from 1 to 10 based on mean accuracy of 10-fold cross-validation.

## 6. Implement the steps (3-5) with the scikit-learn library method and compare it with your previous results.